In [21]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import time, datetime

filePath = '../../data/EURUSD_1.csv'
data = pd.read_csv(filePath)
data['ts'] = pd.to_datetime(data.ts)
print(f'data.shape = {data.shape}')

data.info()

df = data.copy()

"""
fig = go.Figure(data  = [go.Candlestick(
            x=df['ts'],
            open  = df['open'],
            high  = df['high'],
            low   = df['low'],
            close = df['close']
        )])

fig.show()
"""

def hm(value):
    return (np.mean(value) + np.median(value))*0.5

conversion = {
            'open' : 'median',
            'high' : 'max', 
            'low'  : 'min',
            'close': 'mean',
            'vol'  : hm
}

df = df.set_index('ts')
downsampled = df.resample('8H').apply(conversion)
df = df.reset_index()
downsampled = downsampled.reset_index()

#print(downsampled)
print(downsampled[downsampled.ts == '2017-01-02 16:00:00'].open)
print(downsampled[downsampled.ts == '2017-01-02 16:00:00'].close)
print(downsampled[downsampled.ts == '2017-01-04 00:00:00'].high)
print(downsampled[downsampled.ts == '2017-01-04 00:00:00'].low)
print(downsampled[downsampled.ts == '2017-01-05 08:00:00'].vol)

data.shape = (121674, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121674 entries, 0 to 121673
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   open    121674 non-null  float64       
 1   high    121674 non-null  float64       
 2   low     121674 non-null  float64       
 3   close   121674 non-null  float64       
 4   vol     121674 non-null  int64         
 5   ts      121674 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 5.6 MB
1    1.04649
Name: open, dtype: float64
1    1.046375
Name: close, dtype: float64
5    1.04236
Name: high, dtype: float64
5    1.03898
Name: low, dtype: float64
9    481.334375
Name: vol, dtype: float64


/Users/victor/miniconda3/envs/env1/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/victor/miniconda3/envs/env1/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [61]:
df = data.copy()
df = df[df.ts <= '2017-01-03 00:00:00']

df = df.set_index('ts')
upsampled = df.resample(rule='10s').apply(conversion)
df = df.reset_index()
upsampled = upsampled.reset_index()

print(upsampled.open.isna().sum())

uopen = upsampled.interpolate()
print(uopen[uopen.ts == '2017-01-02 09:00:40'].open)

#get index
print(upsampled[upsampled.ts == '2017-01-02 09:00:40'].close)
#looc at index
uclose = upsampled.close.interpolate(method='nearest')
print(uclose)



/Users/victor/miniconda3/envs/env1/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/victor/miniconda3/envs/env1/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


4508
4    1.051683
Name: open, dtype: float64
4   NaN
Name: close, dtype: float64
0       1.05187
1       1.05187
2       1.05187
3       1.05187
4       1.05168
         ...   
5396    1.04540
5397    1.04540
5398    1.04532
5399    1.04532
5400    1.04532
Name: close, Length: 5401, dtype: float64


In [26]:
df = data.copy()
df = df[(df.ts >= '2017-01-05 12:00:00') & (df.ts <= '2017-01-05 13:00:00')]
df['unixtime'] = df.ts.astype(np.int64)

df.head()

ti = np.linspace(df.unixtime.min(), df.unixtime.max(), 135)
upsampled = np.interp(ti, df.unixtime, df.close)

print(upsampled.mean())


1.0502950326146914
